# Searching and Sorting

The purpose of this notebook is to describe various algorithms used for searching and sorting a collection of items that are stored in any of the various data structures. We will first discuss search algorithms before moving on to sorting.

## 1. Searching
Objective is to find an item in a collection of items stored in a list data structure. There are many algorithms that can be used here and a few of them are discussed below.

### 1.1 Sequential Search
Given a list of items, the goal is to ascertain whether an item is in the list or not. The items in the list can be either placed *randomly* (not conforming to any particular ordering) or follow some kind of ordering (arranged in ascending or descending order). In case of a collection wherein the items are *randomly* assigned the time complexity (worst case) is $O(n)$ if the item is present or not because for a collection of length $n$, we will have to compare the item with $n$ items in the collection before we find it or not. However, if the collection comprises an ordered list of items, then in case the item is NOT in the list, we will on *average* have to only traverse $n/2$ items before we figure out that the item is NOT in the list. 

In [128]:
from sys import exit
from time import time

In [1]:
##Unordered List
def sequentialSearchU(aList, item):
    
    found = False
    n = len(aList)
    pos = 0
    while pos < n and not found:
        if aList[pos] == item:
            found = True
            return found, pos
        else:
            pos += 1
    if found == False:
        return found

In [2]:
print(sequentialSearchU([13,2,5,89,10,3], 5))
print(sequentialSearchU([13,2,5,89,10,3], 55))

(True, 2)
False


In [3]:
def sequentialSearchO(aList, item):
    
    stop = False
    found = False
    n = len(aList)
    pos = 0
    while pos < n and not found and not stop:
        if aList[pos] == item:
            found = True
            return found, pos
        elif aList[pos] > item:##Stop since the item is smaller than the item at the current position in the list.
            stop = True
            return found
        else:
            pos += 1
    if found == False:
        return found

In [4]:
print(sequentialSearchO([1,5,10,35,99,120], 4))
print(sequentialSearchO([1,5,10,35,99,120], 120))

False
(True, 5)


### 1.2 Binary Search

Given a *random* collection of items in a list, if we can sort the items in a particular order, then we can take advanatge of the inherent ordering and use a **divide and conquer** strategy to find whether an item is present in the list or not faster. **Binary search** uses a *divide and conquer* strategy to perform faster searches. 

#### Time complexity analysis
Given a list of $n$ items, at each step the number of items to compare our given item with reduces by a factor of half, that is $n/2$. So for example after we make 1 comparison, we're left with $n/2$ items to compare against. After we make the second comparison we're left with $n/4$ (or $n/2^{2}$) items and so on until we reach a point where we've made $i$ comparisons and have only 1 item left. Therefore, $n/2^{i} = 1$. $i$ is the number of comparisons we have to make to get through the entire list and so solving for $i$ we have $i = \log{n}$. Therfore, the time complexity of the binary search algorithm is $O(\log n)$.

In [5]:
def binarySearch(aList, item):
    """This is a recursive implementation of binarySearch."""
    found = False
    first = 0
    last = len(aList)-1
    if len(aList) == 0:##aList[0:0] = empty list
        return False
    else:
        midpoint = len(aList) // 2
        if aList[midpoint] == item:
            found = True
            return found, midpoint
        elif aList[midpoint] > item:##Means item is in the left halve of the list
            last = midpoint - 1
            return binarySearch(aList[first:last], item)
        else:##item is in the right halve of the list
            first = midpoint + 1
            return binarySearch(aList[first:last], item)

testlist = [0, 1, 2, 8, 13, 17, 19, 32, 42,]
print(binarySearch(testlist, 3))
print(binarySearch(testlist, 13))

False
(True, 4)


### 1.3 Hash Tables

A data structure that enables searching for an item with $O(1)$ time complexity. A collection of items are stored in *hash table* (which can be a simple data structure such as a `list` in Python) at individual locations called *slots* that have unique indices associated with them. These indices take on integer values from $0$ to the $m-1$, where $m$ is the length of the table and are initially assigned `None` values. Then given a collection of integers 31, 46 7, 81, 77 and 24 a **hash function** computes the slot locations where these values will be stored so that they can be easily accessed later on. If the length of the hash table $m$ is 11, then this table is said to have a **load factor** $\lambda = \frac{\textrm{number of items stored}}{m} = \frac{6}{11}$. 

A simple example of a hash function is the "remainder function" that computes the slot indices for each item by dividing each item to be stored by $m$ (length of the table) and storing the *remainder*. So for the above integers if $m = 11$, we get $(31, 9), (46, 3), (7, 7), (81, 4), (77, 0), (24, 2)$ as the *slots* or unique indices associated with these integers in the hash table. However, it must be noted that this particular hash function will NOT work if say the next item to be stored is $90$ which would also result in a slot index of $2$ resulting in a **collision**.

Given a collection of items, a hash function that maps each item into a unique slot is referred to as a *perfect hash function*. Unfortunately, given an arbitrary collection of items, there is no systematic way to construct a perfect hash function. One way to always have a perfect hash function is to increase the size of the hash table so that each possible value in the item range can be accommodated. However, this is only possible for a small set of numbers but impractical for larger sets of items. Therefore, the goal is to **create a hash function that minimizes the number of collisions, is easy to compute, and evenly distributes the items in the hash table**.

Other hash functions include the **folding method** wherein the item is divided into (near) equal sized segments, added up together after which the "remainder function" is applied to the sum to create the hash value of the item. Another method called **mid-square** squares the value of the item as a first step and then extracts a certain part (first two or last two or middle two digits) of the squared number and applies the "remainder function" to create the hash. 

Let's look at a simple example of using a hash table to store a string.

In [6]:
def stringHash(string, nh):
    """Takes a string and length of hash table as input
    generates a hash value for the string."""
    
    sm = 0
    for pos in range(len(string)):
        sm = sm + (pos+1)*ord(string[pos])##Use the positions as weights in the sum in order to take care of anagrams.
    
    return sm%nh

In [7]:
s = 'dog'
print(stringHash(s, 11))

4


#### 1.3.1 Collision Resolution

There are many techniques used to resolve **collisions** that occur when the same hash value is generated (using the "remainder function") for multiple items. Some of these are listed below:
* **open addressing with linear probing**: if mutliple items have the same hash value, then sequentially traverse the hash table till a `None` value is found and store the item there. This often results in *clustering* of items with the same hash value around the original slot. Recall that one of the goals of hashing is to *evenly* distribute items in the hash table.
* **linear probing with _skip_**: a common way to deal with clustering is to use a _skip_ value while doing sequential search for the next available slot.
> NOTE: The _skip_ value must be specified in such a way that all slots in the hash table are covered. What is frequently done is to use a _prime_ number for the length of the hash table that works well with _skip_ to visit all the slots in the table. (Sequential search in the worst case will take upto $O(n)$ time, $n$ is the length of the hash table).
* **quadratic probing**: If $h$ is the hash value then while searching for the next available slot, instead of using a constant value of _skip_ we increment it quadratically resulting in $h+1, h+4, h+9, \dots$ as the next set of hash values that will be probed for empty slots.
* **chaining**: storing *references* to the items corresponding to the same hash value (using perhaps a _linked list_ ) at the designated slot. This will help improve the search time for items as well since the number of items corresponding to the same slot value is expected to be much smaller than the length of the entire hash table. This technique also prevents the hash table from growing to very large sizes.

In [58]:
##Implementing the Map Abstract data type using Hashtable class
class HashTable:
    
    def __init__(self, skip):
        
        ##Create a hast table that holds the "keys" whose locations or slots in the table are determined 
        ##using a hashfunction.
        self.size = 11
        self.skip = skip
        self.slots = [None]*self.size
        self.data  = [None]*self.size
    
    def put(self, key, item):
        
        """The goal here is to store (key,value) pairs using a hastable
        (keys) and list data structure (data). The way in which the keys 
        are stored comprises of using a hashfunction to generate the slots
        to store the keys in a list whose indices or hashvalues correlate 
        with those used to store the data values. The goal is to mimic a 
        dictionary in Python wherein the values can easily be accessed using 
        the keys. Hence retrieval has O(1) time complexity.
        """
        ## Generate a hasvalue corresponding to the key
        hashval = self.hashfunc(key, len(self.slots))
        
        ##If the slot or hashvalue is empty, store the key and corresponding 
        ##data value in the slots and data data structures.
        if self.slots[hashval] == None:
            self.slots[hashval] = key
            self.data[hashval] = item
        ##If the slot or hashval is not empty then there are TWO possibilities
        ##1. the slot already contains the given key and just update the data OR
        ##2. the key is different for the same slot meaning collision and so we must
        ##find an empty slot.
        else:
            if self.slots[hashval] == key:
                self.data[hashval] = item
            ##This is the case where the slot is not empty and has already been
            ##assigned a key thereby comprising of a collision. In this case, we
            ##will use a "rehash" function that implements linear probing with a 
            ##skip variable set to 1. That is, go sequentially from one slot to the next 
            ##till we find an empty slot or a slot containing the key in which case we update
            ##the data.
            else:
                ##Get the next slot value
                nextslot = self.rehash(hashval, len(self.slots))
                ##Find next empty slot
                while self.slots[nextslot] != None and self.slots[nextslot] != key:
                    nextslot = self.rehash(nextslot, len(self.slots))
            
                if self.slots[nextslot] == None:
                    self.slots[nextslot] = key
                    self.data[nextslot] = item
                else:
                    self.data[nextslot] = item##Update
        
    ##The "remainder" hash function   
    def hashfunc(self,key,size):
        return key%size
    
    ##The remainder rehash function that uses linear probing with a skip incrementor.
    def rehash(self,oldhash, size):
        return (oldhash+self.skip)%size
    
    ##Get function: Given a key, get the value corresponding to the key.
    def get(self, key):
        """Retrive an item corresponding to the key. If the key is present,
        retrieve the item other wise, linearly probe till you find the key
        and then retrieve the item or return False if the key is not present."""
        #This serves as the intial starting point
        hashval = self.hashfunc(key, len(self.slots))
        found = False
        stop  = False
        data  = None##Return no data
        ##Seacrh the hashtable for the key
        pos = hashval
        while self.slots[pos] != None and not found and not stop:
            ##If current hashval contains the key, then return the data
            if self.slots[pos] == key:
                found = True
                data = self.data[pos]
            else:
                ##Other wise linearly probe through the hashtable.
                pos = self.rehash(pos, len(self.slots))
                if pos == hashval:
                    stop = True##Could not find the item
        
        return data
    
    ##Delete method
    def delete(self, key):
        """The implementation should be straightforward. If hashtable contains the key
        then delete the (key, value) pair from the hashtable."""
        hashval = self.hashfunc(key, len(self.slots))
        found = False
        pos = hashval
        while self.slots[pos] != None and not found:
            if self.slots[pos] == key:
                print('Deleting the key and value pairs.')
                found = True
                self.slots[pos] = None##Delete the key
                self.data[pos] = None##delete the data pertaining to the key 
            else:##Find the key using the rehash logic
                pos = self.rehash(pos, len(self.slots))
                if pos == hashval:
                    try:
                        raise KeyError('Item not found')
                    except KeyError:
                        print('Item not found')
    
    def length(self):
        """Get the length of the hash table."""
        return len(self.slots)
        
    ##Overloading the get, put and delete operators to enable access using []: Don't undertsand this completely.
    def __getitem__(self,key):
        return self.get(key)

    def __setitem__(self,key,data):
        self.put(key,data)
    
    def __delitem__(self, key):
        self.delete(key)
    
    def __len__(self):
        return self.length()

In [54]:
##Instantiate the Hashtable
skip = 3
h = HashTable(skip)
##Load items
h.put(54,"cat")
h.put(37,"dog")
h.put(22, "lion")
h.put(33, "bear")
print(h.slots)
print(h.data)
print(h.get(54))
print(h.get(22))
print(h.get(99))
h.delete(54)
print(h.slots)
print(h.data)
h.delete(99)

[22, None, None, 33, 37, None, None, None, None, None, 54]
['lion', None, None, 'bear', 'dog', None, None, None, None, None, 'cat']
cat
lion
None
Deleting the key and value pairs.
[22, None, None, 33, 37, None, None, None, None, None, None]
['lion', None, None, 'bear', 'dog', None, None, None, None, None, None]


In [59]:
H = HashTable(skip)
H[54] = "cat"
H[37] = "dog"
H[22] = "lion"
H[33] = "bear"
print(H.slots)
print(H.data)
del H[33]
print(H.data)
print(len(H))

[22, None, None, 33, 37, None, None, None, None, None, 54]
['lion', None, None, 'bear', 'dog', None, None, None, None, None, 'cat']
Deleting the key and value pairs.
['lion', None, None, None, 'dog', None, None, None, None, None, 'cat']
11


#### 1.3.2 Complexity analysis
Best case time complexity is $O(1)$ for search using hash tables. However, as the table gets loaded ($\lambda$ increases) more and more collisions start occuring which means more comparisons must be made to find empty slots. For a successful search, the time complexity is $\frac{1}{2}\big(1 + \frac{1}{1- \lambda}\big)$ while for an unsuccessful search (item not present) the time complexity is $\frac{1}{2}\big(1 + \big(\frac{1}{1 - \lambda}\big)^{2}\big)$ when using **open addressing with linear probing**. When using **chaining**, the time complexity is $1 + \frac{\lambda}{2}$ for a successful search and $\lambda$ for an unsuccessful search.

## 2. Sorting

Two things that dictate efficiency of a sorting algorithm are
1. the number of comparisons to be made to ascertain whether the item is larger or smaller than other items
2. the number of exchanges made based on the outcome of the comparisons.

### 2.1 Bubble Sort

Given a collection of items, go through the collection sequentially and continuously compare and "swap" items into their correct locations till all items are in their correct locations. Therefore, if there are $n$ items in a collection, the collection is traversed $n-1$ times till all the items are in the correct order. During each traversal, the largest item in the collectionis put in it's correct place.

In [64]:
def bubbleSort(aList):
    
    ##Get length of the list
    ni = len(aList)-1
    i  = 0
    #At every step the number of passes reduces by 1 as the ith largest item is slotted into place 
    ##resulting in n-i items remaining to compare against.
    for passnum in range(len(aList)-1,0, -1):
        for j in range(passnum):
            if aList[j] > aList[j+1]:
                temp = aList[j]
                aList[j] = aList[j+1]
                aList[j+1] = temp        

In [67]:
alist = [54,26,93,17,77,31,44,55,20]
#bubbleSort(alist)
#print(alist)

The time complexity of this implementation of bubble sort is $O(n^2)$ for the comparisons that need to be made and at most $O(n)$ exchanges (for a completely unsorted list) giving a total time complexity in the worst case of $O(n^2)$. However, an alternate version of this algorithm **stops** when there are *no more exchanges* to be made implying that the list is already sorted.

In [73]:
def shortbubbleSort(aList):
    ##Set exchanges to True to imply that the 
    ##list is unsorted and therefore there will be
    ##exchanges made.
    exchanges = True
    passes = len(aList)-1
    while passes > 0 and exchanges:
        ##Set exchanges to False now until we actually make a comparison followed by
        ##an exchange.
        exchanges = False
        for i in range(passes):
            if aList[i] > aList[i+1]:
                exchanges = True
                temp = aList[i]
                aList[i] = aList[i+1]
                aList[i+1] = temp 
        passes = passes - 1

In [74]:
alist1 = [43,12,56,14,20,35]
shortbubbleSort(alist1)
print(alist1)

[12, 14, 20, 35, 43, 56]


### 2.2 Selection Sort

Extends the Bubble sort idea to carry out only ONE exchange per pass. During each pass, the maximum is found and slotted into place. It takes $n-1$ passes to sort the entire collection and hence the time complexity is $O(n^2)$ but since it swaps only one during each pass it performs faster than regular bubble sort.

In [75]:
def selectionSort(alist):
    ##You need n-1 passes to sort the list completely.
    ##So start with n items (0 to n-1 indices in Python)
    ##initialize MaxPos to 0th index and 
    for fillslot in range(len(alist)-1,0,-1):
        positionOfMax=0
        ##As we traverse the length of the list, 
        ##we compare each element against every other element 
        ##till we find the location of the maximum.
        for location in range(1,fillslot+1):
            if alist[location]>alist[positionOfMax]:
                positionOfMax = location
        
        ##Since fillslot corresponds to the (n-i)th; i=1,2,3,...n-1 passes
        ##position, fill that position with the maximum found
        ##during the ith pass.
        temp = alist[fillslot]
        alist[fillslot] = alist[positionOfMax]
        alist[positionOfMax] = temp

In [76]:
alist2 = [43,12,56,14,20,35]
selectionSort(alist2)
print(alist2)

[12, 14, 20, 35, 43, 56]


### 2.3 Insertion Sort

Again $O(n^2)$ in time complexity but each item in the list is compared against a previously sorted sublist (available to the left of the current position). The given item is compared to each item in the sublist till it's relative position is found where it is subsequently inserted. 

In [84]:
def insertionSort(alist):
    n = len(alist)
    for i in range(1,n):
        currentVal = alist[i]##Let currentValue be value alist[1]
        pos = i##position is also set to 1
        while pos > 0 and alist[pos-1] > currentVal:
            ##Swap the values if currentval is smaller than the one before it.
            alist[pos] = alist[pos-1]##Shifting the value in the current pos to the previous position because it is smaller.
            pos = pos  - 1
        alist[pos] = currentVal     

In [85]:
alist3 = [43,12,56,14,20,35]
insertionSort(alist3)
print(alist3)

[12, 14, 20, 35, 43, 56]


### 2.4 Shell Sort
TO DO

### 2.5 Merge Sort

Uses divide and conquer to sort and merge a collection of items. Merge sort is a **recursive algorithm** that continually splits a list in half. If the list is empty or has one item, it is sorted by definition (the base case). If the list has more than one item, we split the list and recursively invoke a merge sort on both halves. Once the two halves are sorted, the fundamental operation, called a **merge**, is performed. Merging is the process of taking two smaller sorted lists and combining them together into a single, sorted, new list.

Some essential properties of the merge sort algorithm are:
1. It uses "Divide and conquer" strategy to sort a collection of items
2. Recursive -- It recursively splits a collection of items into smaller and smaller sublists which are sorted and then merged.
3. Stable: account for identical values of items and relative ordering of keys in case we have to sort list of tuples.
4. Is not an "in-place" sorting algorithm -- Since it continuously splits a collection into smaller lists which have to be stored in memory, it's space complexity is $O(n)$.

#### 2.5.1 Time and space complexity
Since the merge sort algorithm continuosly splits a collection into smaller and smaller sub-lists till it reaches a point where you only have a single item, the time complexity associated with splitting is $O(\log n)$. Since we also have to merge the sorted $n$ items to produce the final sorted list the time complexity of this algorithm is $O(n\log n)$. 


In [100]:
def mergeSort(alist, start,end):
    print("Splitting ",alist)
    if len(alist)>1:
        mid = len(alist)//2
        start = 0
        end   = len(alist)
        ##Remove the slice operator because it is of O(k); k is the length of the slice.
        lefthalf = alist[start:mid]
        righthalf = alist[mid:end]
        print("Left half (before) = ",lefthalf)
        print("Right half (before)=", righthalf)
        mergeSort(lefthalf, start, end)
        mergeSort(righthalf, start, end)

        i=0
        j=0
        k=0
        
        print("Left half (after) = ",lefthalf)
        print("Right half (after) =", righthalf)
        ##This is the merge part that merges the sorted results from left and right halves by updating 
        ##alist.
        while i < len(lefthalf) and j < len(righthalf):
            if lefthalf[i] <= righthalf[j]:##To accomodate for identical items we have the equal to sign.
                alist[k]=lefthalf[i]
                i=i+1
            else:
                alist[k]=righthalf[j]
                j=j+1
            k=k+1
        ##This is to populate aList with the items remaining in lefthalf 
        while i < len(lefthalf):
            alist[k]=lefthalf[i]
            i=i+1
            k=k+1
        ##This is to populate aList with the items remaining in righthalf 
        while j < len(righthalf):
            alist[k]=righthalf[j]
            j=j+1
            k=k+1
    print("Merging ",alist)

In [123]:
alist = [54,26,93,17,77,31,44,55,20]

start = 0
end = len(alist)
startt = time()
mergeSort(alist, start, end)
print('End: ', time()-startt)
print(alist)

Splitting  [54, 26, 93, 17, 77, 31, 44, 55, 20]
Left half (before) =  [54, 26, 93, 17]
Right half (before)= [77, 31, 44, 55, 20]
Splitting  [54, 26, 93, 17]
Left half (before) =  [54, 26]
Right half (before)= [93, 17]
Splitting  [54, 26]
Left half (before) =  [54]
Right half (before)= [26]
Splitting  [54]
Merging  [54]
Splitting  [26]
Merging  [26]
Left half (after) =  [54]
Right half (after) = [26]
Merging  [26, 54]
Splitting  [93, 17]
Left half (before) =  [93]
Right half (before)= [17]
Splitting  [93]
Merging  [93]
Splitting  [17]
Merging  [17]
Left half (after) =  [93]
Right half (after) = [17]
Merging  [17, 93]
Left half (after) =  [26, 54]
Right half (after) = [17, 93]
Merging  [17, 26, 54, 93]
Splitting  [77, 31, 44, 55, 20]
Left half (before) =  [77, 31]
Right half (before)= [44, 55, 20]
Splitting  [77, 31]
Left half (before) =  [77]
Right half (before)= [31]
Splitting  [77]
Merging  [77]
Splitting  [31]
Merging  [31]
Left half (after) =  [77]
Right half (after) = [31]
Merging 

### 2.6 Quick Sort
Builds on merge sort without the additional memory requirment. Instead of splitting the list at the midpoint into multiple sublists that require storage, quick sort tries to find an optimal **pivot** point to split the list into smaller sublists (that are already partially sorted as part of the partitioning or splitting process). The quick sort function is then recursively called to further sort these sublists. The selection of the split point is key to the performance of this algorithm. In the current implementation the first value in the list is taken as the split point. However since a split point should ideally be located at the center of the list, for a semi-sorted list picking the first (smallest) value or last (largest) value as a pivot or split point can greatly decrease the efficiency of this algorithm. 

In the best case the time complexity of this algorithm is $O(n\log n)$ but in the worst case (described above) it is $O(n^2)$.

In [131]:
##Quick Sort algorithm.
def quickSort(alist,first,last):
    print(first,last)
    if first < last:##Last can change here based on the splitpoint.
        ##To find the split point to be used to partition the list,
        ##supply the list the start and end positions of the list. 
        splitpoint = partition(alist,first,last)
        
        ##Split the list at it's splitpoint and 
        ##recursively call QuickSort to sort the sublists
        quickSort(alist, first, splitpoint-1)
        quickSort(alist, splitpoint+1, last)

def partition(alist,first,last):
    
    ##Choose an appropriate split value. In this implementation, the first
    ##value of the list is chosen as the split value.
    #split = alist[first]
    ##Choose median of three as the split value
    split = pickMedian([alist[first], alist[len(alist)//2], alist[last]])
    print(split)
    ##Flag to indicate that the pivot or split point location has been found.
    done = False
    left  = first + 1
    right = last
    
    ##Start iterating
    while not done:
        ##As long as the leftmark is smaller than the rightmark and the split value is 
        ##larger than the value corresponding to the leftmark, keep incrementing leftmark till we
        ##encounter a value that is larger than our pivot value.
        while left <= right and alist[left] <= split:
            left = left + 1
        ##As long as the rightmark is greater than the leftmark and the split value is 
        ##smaller than the value corresponding to the rightmark, keep decrementing rightmark till we
        ##encounter a value that is smaller than our pivot value.
        while alist[right] >= split and right >= left:
            right = right-1
        print(alist[left],alist[right])
        if right < left:##We found the splitvalue= rightmark
            done = True
        else:
            ##Swap the leftmark and rightmark values and proceed.
            temp = alist[left]
            alist[left] = alist[right]
            alist[right] = temp
    
    ##Once we've found our split point, swap the pivot value (first value in the list here) with the rightmark value.
    ##This ensures that all values to the left of the pivot point are smaller than the pivot value
    ##and all values to the right of the pivot value are larger than it.
    temp = alist[first]
    alist[first] = alist[right]
    alist[right]= temp
    
    return right  

##Median of three
def pickMedian(l):
    
    ##Use bubble sort to sort the values first: O(n^2)
    print('Before', l)
    n = len(l)-1
    for passes in range(n,0,-1):
        for j in range(passes):
            if l[j] >= l[j+1]:
                temp = l[j+1]
                l[j+1] = l[j]
                l[j] = temp
    print('After',l)
    return l[len(l)//2]

In [132]:
alist = [54,26,93,17,77,31,44,55,20]
first = 0
last  = len(alist)-1
start = time()
quickSort(alist, first, last)
print('End', time()-start)
print(alist)

0 8
Before [54, 77, 20]
After [20, 54, 77]
54
93 20
77 44
77 31
0 4
Before [31, 44, 44]
After [31, 44, 44]
44
54 44
0 3
Before [44, 31, 17]
After [17, 31, 44]
31
31 17
0 2
Before [17, 31, 20]
After [17, 20, 31]
20
26 17
0 -1
1 2
Before [26, 31, 20]
After [20, 26, 31]
26
44 20
1 1
3 2
4 3
5 4
6 8
Before [77, 31, 93]
After [31, 77, 93]
77
93 55
6 6
8 8
End 0.0019943714141845703
[17, 20, 26, 44, 31, 54, 55, 77, 93]
